Berikut adalah script untuk mengubah pdf lampiran profil kesehatan Jawa Barat jadi csv
Caveat: script setiap tahun berbeda karena halaman dan kerangka tabel berbeda-beda setiap tahunnya. Sehingga tidak dapat dilakukan automasi.
Feel free to give me any suggestions

Cara run </br>
python export_data_to_csv.py index_halaman_start index_halaman_end

Misal </br>
python export_data_to_csv.py 0 100

In [249]:
# import library
import tabula
import camelot
import pandas as pd
import datetime as dt
import sys

pd.options.display.max_columns = None

## Scraping Data

In [159]:
# Tenaga Puskemas

df = tabula.read_pdf("2022.pdf", pages="56")[0]
df = df.iloc[:,1:]
df.rename(columns={'Jenis Tenaga Kesehatan': "Jenis Tenaga", "Jumlah Tenaga Kesehatan" : "Jumlah Tenaga"}, inplace = True)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(str)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")

df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(int)
df['tahun'] = "2022"
df['unit_kerja'] = "puskesmas"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Jenis Tenaga   9 non-null      object
 1   Jumlah Tenaga  9 non-null      int32 
 2   tahun          9 non-null      object
 3   unit_kerja     9 non-null      object
dtypes: int32(1), object(3)
memory usage: 380.0+ bytes


<ipython-input-159-dcd7190f4afd>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")


In [53]:
# Tenaga Rumah Sakit
df = tabula.read_pdf("2022.pdf", pages="60")[0]
df.dropna(inplace=True)
df.rename(columns={'Jenis Tenaga Kesehatan': "Jenis Tenaga", "Jumlah Tenaga Kesehatan" : "Jumlah Tenaga"}, inplace = True)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(str)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")

df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(int)
df['tahun'] = "2022"
df['unit_kerja'] = "rumah sakit"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Jenis Tenaga   9 non-null      object
 1   Jumlah Tenaga  9 non-null      int32 
 2   tahun          9 non-null      object
 3   unit_kerja     9 non-null      object
dtypes: int32(1), object(3)
memory usage: 324.0+ bytes


<ipython-input-53-9a5b700f41fe>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")


In [52]:
## Rekapitulasi Rasio Tenaga Kesehatan /100.000
df = tabula.read_pdf("2022.pdf", pages="61")[1]
df = df.iloc[3:,1:3]
df.set_axis(["jenis_tenaga","Ratio/100.000 Penduduk"], axis=1, inplace=True)
df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].str.replace(',','.')
df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].astype(float)
df.replace("Tenaga\rTeknisKefarmasian", "Tenaga Teknis Kefarmasian", inplace=True)

df2 = tabula.read_pdf("2022.pdf", pages="62")[0]
header_value = list(df2.columns)
df2 = df2.append(pd.Series(header_value, index=df2.columns), ignore_index=True)
df2 = df2.iloc[:,1:3]
df2.set_axis(["jenis_tenaga","Ratio/100.000 Penduduk"], axis=1, inplace=True)
df2['Ratio/100.000 Penduduk'] = df2['Ratio/100.000 Penduduk'].str.replace(',','.')
df2['Ratio/100.000 Penduduk'] = df2['Ratio/100.000 Penduduk'].astype(float)
df2

df = pd.concat([df,df2], ignore_index = True)
df['tahun'] = "2022"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   jenis_tenaga            13 non-null     object 
 1   Ratio/100.000 Penduduk  13 non-null     float64
 2   tahun                   13 non-null     object 
dtypes: float64(1), object(2)
memory usage: 440.0+ bytes


<ipython-input-52-3861e0b73a98>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.Series(header_value, index=df2.columns), ignore_index=True)


In [174]:
## Jumlah Fasilitas Kesehatan Menurut Kepemilikan 
### Data ini dicek secara manual pada setiap laporan kesehatan per kab/kota 2022

data = {
    'Kabupaten/Kota': ['KABUPATEN BOGOR', 'KABUPATEN SUKABUMI', 'KABUPATEN CIANJUR', 'KABUPATEN BANDUNG', 'KABUPATEN GARUT',
                            'KABUPATEN TASIKMALAYA', 'KABUPATEN CIAMIS', 'KABUPATEN KUNINGAN', 'KABUPATEN CIREBON',
                            'KABUPATEN MAJALENGKA', 'KABUPATEN SUMEDANG', 'KABUPATEN INDRAMAYU', 'KABUPATEN SUBANG',
                            'KABUPATEN PURWAKARTA', 'KABUPATEN KARAWANG', 'KABUPATEN BEKASI', 'KABUPATEN BANDUNG BARAT',
                            'KABUPATEN PANGANDARAN', 'KOTA BOGOR', 'KOTA SUKABUMI', 'KOTA BANDUNG', 'KOTA CIREBON',
                            'KOTA BEKASI', 'KOTA DEPOK', 'KOTA CIMAHI', 'KOTA TASIKMALAYA', 'KOTA BANJAR'],
    'Jumlah': [298, 31, 0, 13, 6, 10, 8, 63, 251, 51, 27, 77, 13, 0, 276, 66, 0, 3, 75, 40, 0, 45, 66, 131, 69, 27, 16]
}

df = pd.DataFrame(data)
df['tahun'] = "2022"

,Kabupaten/Kota,Jumlah,tahun
0,KABUPATEN BOGOR,298,2022
1,KABUPATEN SUKABUMI,31,2022
2,KABUPATEN CIANJUR,0,2022
3,KABUPATEN BANDUNG,13,2022
4,KABUPATEN GARUT,6,2022
5,KABUPATEN TASIKMALAYA,10,2022
6,KABUPATEN CIAMIS,8,2022
7,KABUPATEN KUNINGAN,63,2022
8,KABUPATEN CIREBON,251,2022
9,KABUPATEN MAJALENGKA,51,2022


In [200]:
### PELAYANAN KESEHATAN GIGI DAN MULUT MENURUT KECAMATAN DAN PUSKESMAS

tables = camelot.read_pdf("2022.pdf", flavor='stream', pages='288')

df = tables[0].df
df.columns = df.iloc[5]

df = df.iloc[6:,[1,3,5,9,11,13,15] ]
df.reset_index(drop=True, inplace=True)
df.iloc[27,0] = "Jawa Barat"
df.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan", "Jumlah Kasus Gigi", "Jumlah Kasus dirujuk", "% Rasio Dirujuk"], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')
for col in ['Rasio Pencabutan', '% Rasio Dirujuk']:
    df[col] = df[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap', 'Jumlah Kasus Gigi', 'Jumlah Kasus dirujuk']:
    df[col] = df[col].str.replace('.', '').astype(int)
df['tahun'] = "2022"
df['unit_kerja'] = "puskesmas"
df

<ipython-input-200-6f9854fba6ad>:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '').astype(int)


,Kabupaten/Kota,Jumlah Tumpatan Gigi Tetap,Pencabutan Gigi Tetap,Rasio Pencabutan,Jumlah Kasus Gigi,Jumlah Kasus dirujuk,% Rasio Dirujuk,tahun,unit_kerja
0,Kabupaten Bogor,10905,11277,0.97,161427,5016,3.11,2022,puskesmas
1,Kabupaten Sukabumi,4712,6911,0.68,61684,2723,4.41,2022,puskesmas
2,Kabupaten Cianjur,2022,3577,0.57,27559,1269,4.60,2022,puskesmas
3,Kabupaten Bandung,19783,20738,0.95,210294,9846,4.68,2022,puskesmas
4,Kabupaten Garut,4241,7070,0.60,50119,889,1.77,2022,puskesmas
5,Kabupaten Tasikmalaya,914,1131,0.81,9813,382,3.89,2022,puskesmas
6,Kabupaten Ciamis,1487,3096,0.48,32949,6040,18.33,2022,puskesmas
7,Kabupaten Kuningan,1466,2961,0.50,27405,3029,11.05,2022,puskesmas
8,Kabupaten Cirebon,5039,7660,0.66,114658,6551,5.71,2022,puskesmas
9,Kabupaten Majalengka,477,1906,0.25,14933,1869,12.52,2022,puskesmas


In [251]:
## PELAYANAN KESEHATAN GIGI DAN MULUT PADA ANAK SD DAN SETINGKAT MENURUT JENIS KELAMIN, KECAMATAN, DAN PUSKESMAS

tables = camelot.read_pdf("2022.pdf", flavor='stream', pages='289')

df = tables[0].df
# df.columns = df.iloc[7]

df = df.iloc[9:, [1, 2, 4, 5] + list(range(7, df.shape[1])) ]
# ### Replace #### Value
df.iloc[df[1] == 'Kabupaten Sumedang', 1:] = 0

df.reset_index(drop=True, inplace=True)
df = df.rename_axis(None, axis=1)
df.iloc[27,0] = "Jawa Barat"
df.set_axis(['Kabupaten/Kota', 'jumlah_SD/MI','jumlah_SD/MI_SG_Massal','%jumlah_SD/MI_SG_Massal',
             'jumlah_SD/MI_LG','%jumlah_SD/MI_LG',
            'jumlah_murid_SD/MI_L', 'jumlah_murid_SD/MI_P', 'jumlah_murid_SD/MI_LP',
            'jumlah_murid_SD/MI_L_Diperiksa','%jumlah_murid_SD/MI_L_Diperiksa','jumlah_murid_SD/MI_P_Diperiksa','%jumlah_murid_SD/MI_P_Diperiksa', 'jumlah_murid_SD/MI_LP_Diperiksa','%jumlah_murid_SD/MI_LP_Diperiksa',
            'jumlah_murid_SD/MI_L_PR','jumlah_murid_SD/MI_P_PR','jumlah_murid_SD/MI_LP_PR',
            'jumlah_murid_SD/MI_L_MP','%jumlah_murid_SD/MI_L_MP','jumlah_murid_SD/MI_P_MP','%jumlah_murid_SD/MI_P_MP','jumlah_murid_SD/MI_LP_MP','%jumlah_murid_SD/MI_LP_MP'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')
# Additional string for another Kabupaten
additional_values_str = ['554', '218', '39.4', '1148', '526.6', '37727', '39390', '77117', '20100', '53.3', '19494', '49.5', '28154', '36.5', '5280', '5408', '10688', '3005', '56.9', '2913', '53.9', '5918', '55.4']

# Replace commas with dots
additional_values_str = [value.replace(',', '.') for value in additional_values_str]

df.loc[df['Kabupaten/Kota'] == 'Kabupaten Sumedang', df.columns[1:]] = additional_values_str

## Replace ####
for col in df.columns:
    df.loc[df[col].astype(str).str.contains('#'), col] = 0

for col in df.columns[df.columns.str.contains('%')]:
    df[col] = df[col].str.replace(',', '.').astype(float)

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kabupaten/Kota'))]:
    df[col] = df[col].replace('\.', '', regex=True).astype(int)

    
df = df.fillna(0)
#### Fix #### Value Before
df.loc[df["Kabupaten/Kota"] == 'Kabupaten Indramayu', "jumlah_murid_SD/MI_LP"] = df.loc[df["Kabupaten/Kota"] == 'Kabupaten Indramayu', "jumlah_murid_SD/MI_L"] + df.loc[df["Kabupaten/Kota"] == 'Kabupaten Indramayu', "jumlah_murid_SD/MI_P"]
df.loc[df["Kabupaten/Kota"] == 'Kabupaten Indramayu', "%jumlah_murid_SD/MI_LP_Diperiksa"] = round(df.loc[df["Kabupaten/Kota"] == 'Kabupaten Indramayu', "jumlah_murid_SD/MI_LP_Diperiksa"] / df.loc[df["Kabupaten/Kota"] == 'Kabupaten Indramayu', "jumlah_murid_SD/MI_LP"],2)
df.at[27, 'jumlah_murid_SD/MI_L'] = df.iloc[: 26]['jumlah_murid_SD/MI_L'].sum()
df.at[27, 'jumlah_murid_SD/MI_P'] = df.iloc[: 26]['jumlah_murid_SD/MI_P'].sum()
df.at[27, 'jumlah_murid_SD/MI_LP'] = df.iloc[: 26]['jumlah_murid_SD/MI_LP'].sum()
df.at[27, 'jumlah_murid_SD/MI_L_PR'] = df.iloc[: 26]['jumlah_murid_SD/MI_L_PR'].sum()
df.at[27, 'jumlah_murid_SD/MI_P_PR'] = df.iloc[: 26]['jumlah_murid_SD/MI_P_PR'].sum()
df.at[27, 'jumlah_murid_SD/MI_LP_PR'] = df.iloc[: 26]['jumlah_murid_SD/MI_LP_PR'].sum()
df.at[27, 'jumlah_murid_SD/MI_L_MP'] = df.iloc[: 26]['jumlah_murid_SD/MI_L_MP'].sum()
df.at[27, 'jumlah_murid_SD/MI_P_MP'] = df.iloc[: 26]['jumlah_murid_SD/MI_P_MP'].sum()
df.at[27, 'jumlah_murid_SD/MI_LP_MP'] = df.iloc[: 26]['jumlah_murid_SD/MI_LP_MP'].sum()
tahun = "2022"
df['tahun'] = tahun
df.to_csv(f"dataset/pelayanan_gigi_anaksd/{tahun}.csv", index=False)
df

,Kabupaten/Kota,jumlah_SD/MI,jumlah_SD/MI_SG_Massal,%jumlah_SD/MI_SG_Massal,jumlah_SD/MI_LG,%jumlah_SD/MI_LG,jumlah_murid_SD/MI_L,jumlah_murid_SD/MI_P,jumlah_murid_SD/MI_LP,jumlah_murid_SD/MI_L_Diperiksa,%jumlah_murid_SD/MI_L_Diperiksa,jumlah_murid_SD/MI_P_Diperiksa,%jumlah_murid_SD/MI_P_Diperiksa,jumlah_murid_SD/MI_LP_Diperiksa,%jumlah_murid_SD/MI_LP_Diperiksa,jumlah_murid_SD/MI_L_PR,jumlah_murid_SD/MI_P_PR,jumlah_murid_SD/MI_LP_PR,jumlah_murid_SD/MI_L_MP,%jumlah_murid_SD/MI_L_MP,jumlah_murid_SD/MI_P_MP,%jumlah_murid_SD/MI_P_MP,jumlah_murid_SD/MI_LP_MP,%jumlah_murid_SD/MI_LP_MP,tahun
0,Kabupaten Bogor,1699,1699,100.0,1699,100.0,119461,108524,227985,7504,6.3,7322,6.7,14826,6.50,3212,2766,5978,423,13.2,312,11.3,735,12.3,2022
1,Kabupaten Sukabumi,3137,659,21.0,2855,91.0,93415,89119,182534,54479,58.3,48836,54.8,103315,56.60,18213,18213,36426,8061,44.3,8279,45.5,16340,44.9,2022
2,Kabupaten Cianjur,1240,25,2.0,124,10.0,129865,121577,251442,1120,0.9,1360,1.1,2480,1.00,400,365,765,380,95.0,320,87.7,700,91.5,2022
3,Kabupaten Bandung,1601,1097,68.5,1458,91.1,202013,191490,393503,26185,13.0,25337,13.2,51522,13.10,8193,8127,16320,1402,17.1,1422,17.5,2824,17.3,2022
4,Kabupaten Garut,2270,467,20.6,1461,64.4,150200,184201,334401,31258,20.8,46887,25.5,78145,23.40,12673,19011,31684,2654,20.9,3982,20.9,6636,20.9,2022
5,Kabupaten Tasikmalaya,2736,135,4.9,1866,68.2,52605,13920,66525,2005,3.8,1250,9.0,3255,4.90,947,895,1842,687,72.5,563,62.9,1250,67.9,2022
6,Kabupaten Ciamis,621,342,55.1,564,90.8,46163,69243,115406,24151,52.3,36226,52.3,60377,52.30,7146,10722,17868,2718,38.0,4077,38.0,6795,38.0,2022
7,Kabupaten Kuningan,657,282,42.9,634,96.5,34188,32660,66848,26412,77.3,25305,77.5,51717,77.40,6874,6863,13737,2493,36.3,2682,39.1,5175,37.7,2022
8,Kabupaten Cirebon,1093,1033,94.5,1047,95.8,65883,58393,124276,22332,33.9,42298,72.4,64630,52.00,11533,11489,23022,5657,49.1,5682,49.5,11339,49.3,2022
9,Kabupaten Majalengka,762,580,76.1,752,98.7,40907,39429,80336,25157,61.5,25262,64.1,50419,62.80,4937,5015,9952,3081,62.4,3296,65.7,6377,64.1,2022


In [188]:
## Jumlah Tenaga Medis Per Kab/Kota Puskesmaas
tables = camelot.read_pdf("2022.pdf", flavor='stream', pages='238')

df = tables[1].df

df = df.iloc[10:, [1] + list(range(11, 17))]

# df = df.iloc[9: ]
df.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kab/Kota'))]:
    df[col] = df[col].str.replace(',', '').astype(int)
    
df['Kab/Kota'] = df['Kab/Kota'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

,Kab/Kota,jml_dg_l,jml_dg_p,jml_dg_lp,jml_dgsp_l,jml_dgsp_p,jml_dgsp_lp
10,Kabupaten Bogor,5,69,74,0,0,0
11,Kabupaten Sukabumi,11,21,32,0,0,0
12,Kabupaten Cianjur,9,27,36,0,0,0
13,Kabupaten Bandung,11,61,72,0,0,0
14,Kabupaten Garut,6,21,27,0,0,0
15,Kabupaten Tasikmalaya,9,18,27,0,0,0
16,Kabupaten Ciamis,3,14,17,0,0,0
17,Kabupaten Kuningan,8,16,24,0,0,0
18,Kabupaten Cirebon,12,43,55,0,0,0
19,Kabupaten Majalengka,7,17,24,0,0,0


In [189]:
## Jumlah Tenaga Medis Per Kab/Kota Puskesmaas
tables = camelot.read_pdf("2022.pdf", flavor='stream', pages='239')

df2 = tables[0].df

df2 = df2.iloc[:15, [0] + list(range(10, 16))]

df2 = df2.iloc[5: ]
df2.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df2 = df2.replace('', '0')
df2 = df2.replace('-', '0')

for col in df.columns[~(df2.columns.str.contains('%') | (df2.columns == 'Kab/Kota'))]:
    df2[col] = df2[col].str.replace(',', '').astype(int)
    
df2['Kab/Kota'] = df2['Kab/Kota'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

df = pd.concat([df, df2], ignore_index=True)


In [190]:
# Menggabungkan kolom 'jml_dg_l', 'jml_dg_p', 'jml_dgsp_l', dan 'jml_dgsp_p' menjadi satu kolom 'jumlah'
df_dg = pd.concat([
    df.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='puskesmas')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_pks = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

,Kab/Kota,jumlah,jns_kelamin,profesi,unit_kerja
3,Kabupaten Bandung,11,laki-laki,dokter gigi,puskesmas
57,Kabupaten Bandung,0,laki-laki,dokter gigi spesialis,puskesmas
30,Kabupaten Bandung,61,perempuan,dokter gigi,puskesmas
84,Kabupaten Bandung,0,perempuan,dokter gigi spesialis,puskesmas
16,Kabupaten Bandung Barat,7,laki-laki,dokter gigi,puskesmas
...,...,...,...,...,...
100,Kota Sukabumi,0,perempuan,dokter gigi spesialis,puskesmas
25,Kota Tasikmalaya,2,laki-laki,dokter gigi,puskesmas
79,Kota Tasikmalaya,0,laki-laki,dokter gigi spesialis,puskesmas
52,Kota Tasikmalaya,9,perempuan,dokter gigi,puskesmas


In [191]:
## Jumlah Tenaga Medis Kab/Kota Rumah Sakit

tables = camelot.read_pdf("2022.pdf", flavor='stream', pages='239')

df = tables[0].df

df = df.iloc[:45, [0] + list(range(10, 16))]

df = df.iloc[18: ]
df.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kab/Kota'))]:
    df[col] = df[col].str.replace(',', '').astype(int)

df['Kab/Kota'] = df['Kab/Kota'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

,Kab/Kota,jml_dg_l,jml_dg_p,jml_dg_lp,jml_dgsp_l,jml_dgsp_p,jml_dgsp_lp
18,Kabupaten Bogor,17,62,79,23,32,55
19,Kabupaten Sukabumi,6,15,21,3,1,4
20,Kabupaten Cianjur,1,8,9,1,3,4
21,Kabupaten Bandung,7,21,28,2,13,15
22,Kabupaten Garut,1,7,8,0,2,2
23,Kabupaten Tasikmalaya,9,18,27,0,0,0
24,Kabupaten Ciamis,2,6,8,1,3,4
25,Kabupaten Kuningan,5,12,17,0,2,2
26,Kabupaten Cirebon,3,14,17,7,9,16
27,Kabupaten Majalengka,1,6,7,1,0,1


In [192]:
df_dg = pd.concat([
    df.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='rumah sakit')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_rs = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

In [193]:
# ## Klinik DIKLAT KESEHATAN
# _garut = pd.DataFrame({
#     'Kab/Kota': ['Kabupaten Garut'],
#     'jml_dg_l': [4],
#     'jml_dg_p': [7],
#     'jml_dgsp_l': [1],
#     'jml_dgsp_p': [0]
# })

# df_bekasi = pd.DataFrame({
#     'Kab/Kota': ['Kabupaten Bekasi'],
#     'jml_dg_l': [31],
#     'jml_dg_p': [139],
#     'jml_dgsp_l': [11],
#     'jml_dgsp_p': [28]
# })

# df_combined = pd.concat([df_garut, df_bekasi], ignore_index=True)

# df_dg = pd.concat([
#     df_combined.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='klinik diklat kesehatan'),
#     df_combined.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='klinik diklat kesehatan'),
#     df_combined.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='klinik diklat kesehatan'),
#     df_combined.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='klinik diklat kesehatan')
# ], ignore_index=True)

# # Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
# df_dg = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])
# df_dg

In [194]:
## Klinik Dinas Kesehatan
data_karawang = {'Kab/Kota': 'Kabupaten Karawang',
              'jml_dg_l': 0,
              'jml_dg_p': 1,
              'jml_dgsp_l': 0,
              'jml_dgsp_p': 0}

data_purwakarta = {'Kab/Kota': 'Kota Purwakarta',
                'jml_dg_l': 0,
                'jml_dg_p': 1,
                'jml_dgsp_l': 0,
                'jml_dgsp_p': 0}

df_karawang = pd.DataFrame([data_karawang])
df_purwakarta = pd.DataFrame([data_purwakarta])

df_combined = pd.concat([df_karawang, df_purwakarta], ignore_index=True)

df_dg = pd.concat([
    df_combined.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='klinik DINKES')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_dk = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])
df_dg = pd.concat([df_dg_rs,df_dg_pks,df_dg_dk], ignore_index=True )
tahun = "2022"
df_dg['tahun'] = tahun

,Kab/Kota,jumlah,jns_kelamin,profesi,unit_kerja,tahun
0,Kabupaten Bandung,7,laki-laki,dokter gigi,rumah sakit,2022
1,Kabupaten Bandung,2,laki-laki,dokter gigi spesialis,rumah sakit,2022
2,Kabupaten Bandung,21,perempuan,dokter gigi,rumah sakit,2022
3,Kabupaten Bandung,13,perempuan,dokter gigi spesialis,rumah sakit,2022
4,Kabupaten Bandung Barat,5,laki-laki,dokter gigi,rumah sakit,2022
...,...,...,...,...,...,...
219,Kabupaten Karawang,0,perempuan,dokter gigi spesialis,klinik DINKES,2022
220,Kota Purwakarta,0,laki-laki,dokter gigi,klinik DINKES,2022
221,Kota Purwakarta,0,laki-laki,dokter gigi spesialis,klinik DINKES,2022
222,Kota Purwakarta,1,perempuan,dokter gigi,klinik DINKES,2022


In [197]:
df_dg.to_csv(f'dataset/dg_unit_kerja/{tahun}.csv', index=False)
print(f"Job Scraping pada Profil Kesehatan Jawa Barat tahun {tahun} sudah selesai")

Job Scraping pada Profil Kesehatan Jawa Barat tahun 2022 sudah selesai


In [246]:
tables = camelot.read_pdf("https://diskes.jabarprov.go.id/assets/unduhan/743ed67d9392011b54baee048ef56e05.pdf", flavor='stream', pages='277')

df = tables[0].df
# df.columns = df.iloc[7]

df = df.iloc[9:, [2, 4, 6,7,9] + list(range(10, df.shape[1])) ]
# # ### Replace #### Value
df.iloc[df[2] == 'CIMANGGUNG', 5] = "75.0"
df.iloc[df[2] == 'CIMANGGUNG', 6] = "3437"
df.iloc[df[2] == 'CIMANGGUNG', 12] = "49.7"
df.iloc[df[2] == 'CIMANGGUNG', 13] = "3310"
df.iloc[df[2] == 'MARGAJAYA', 12] = "32.4"
df.iloc[df[2] == 'MARGAJAYA', 13] = "1626"
df.iloc[df[2] == 'SUKASARI', 5] = "83.3"
df.iloc[df[2] == 'SUKASARI', 6] = "1071"
df.iloc[df[2] == 'SUKASARI', 12] = "42.1"
df.iloc[df[2] == 'SUKASARI', 13] = "1109"
df.iloc[df[2] == 'HAURNGOMBONG', 12] = "34.4"
df.iloc[df[2] == 'HAURNGOMBONG', 13] = "1108"
df.iloc[df[2] == 'PAMULIHAN', 5] = "85.7"
df.iloc[df[2] == 'PAMULIHAN', 6] = "1127"
df.iloc[df[2] == 'PADASUKA', 12] = "34.4"
df.iloc[df[2] == 'PADASUKA', 13] = "1108"
df.iloc[df[2] == 'SUMEDANG SELATANSUMEDANG SELATAN', 12] = "34.4"
df.iloc[df[2] == 'SUMEDANG SELATANSUMEDANG SELATAN', 13] = "1108"
df.iloc[df[2] == 'SUMEDANG SELATANSUMEDANG SELATAN', 21] = "77.7"
df.iloc[df[2] == 'SUMEDANG SELATANSUMEDANG SELATAN', 22] = "1107"
df.iloc[11,0] = "KOTAKALER "
df.iloc[11, 12] = "94.5"
df.iloc[11, 13] = "1474"
df.iloc[df[2] == 'GANEAS', 12] = "34.4"
df.iloc[df[2] == 'GANEAS', 13] = "1108"
df.iloc[df[2] == 'SITURAJA', 12] = "34.4"
df.iloc[df[2] == 'SITURAJA', 13] = "1108"
df.iloc[df[2] == 'DARMARAJA', 12] = "34.4"
df.iloc[df[2] == 'DARMARAJA', 13] = "1108"
df.iloc[df[2] == 'JATINUNGGAL', 12] = "34.4"
df.iloc[df[2] == 'JATINUNGGAL', 13] = "1108"
df.iloc[df[2] == 'UJUNGJAYA', 12] = "34.4"
df.iloc[df[2] == 'UJUNGJAYA', 13] = "1108"
df.iloc[df[2] == 'CONGGEANG', 12] = "34.4"
df.iloc[df[2] == 'CONGGEANG', 13] = "1108"
df.iloc[df[2] == 'CIMALAKA', 12] = "34.4"
df.iloc[df[2] == 'CIMALAKA', 13] = "1108"


df.reset_index(drop=True, inplace=True)
df = df.rename_axis(None, axis=1)
df.set_axis(['Kabupaten/Kota', 'jumlah_SD/MI','jumlah_SD/MI_SG_Massal','%jumlah_SD/MI_SG_Massal',
             'jumlah_SD/MI_LG','%jumlah_SD/MI_LG',
            'jumlah_murid_SD/MI_L', 'jumlah_murid_SD/MI_P', 'jumlah_murid_SD/MI_LP',
            'jumlah_murid_SD/MI_L_Diperiksa','%jumlah_murid_SD/MI_L_Diperiksa','jumlah_murid_SD/MI_P_Diperiksa','%jumlah_murid_SD/MI_P_Diperiksa', 'jumlah_murid_SD/MI_LP_Diperiksa','%jumlah_murid_SD/MI_LP_Diperiksa',
            'jumlah_murid_SD/MI_L_PR','jumlah_murid_SD/MI_P_PR','jumlah_murid_SD/MI_LP_PR',
            'jumlah_murid_SD/MI_L_MP','%jumlah_murid_SD/MI_L_MP','jumlah_murid_SD/MI_P_MP','%jumlah_murid_SD/MI_P_MP','jumlah_murid_SD/MI_LP_MP','%jumlah_murid_SD/MI_LP_MP'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

df = df.iloc[:-1]
df.iloc[34,0] = "Jawa Barat"
df.iloc[34, 19] = "56.9"
df.iloc[34, 20] = "1108"
df.iloc[34, 22] = "3213"
## Replace ####
for col in df.columns:
    df.loc[df[col].astype(str).str.contains('#'), col] = 0

for col in df.columns[df.columns.str.contains('%')]:
    df[col] = df[col].str.replace(',', '.').astype(float)

df.fillna(0,inplace=True)
for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kabupaten/Kota'))]:
    df[col] = df[col].replace('\,', '', regex=True).astype(int)

df

,Kabupaten/Kota,jumlah_SD/MI,jumlah_SD/MI_SG_Massal,%jumlah_SD/MI_SG_Massal,jumlah_SD/MI_LG,%jumlah_SD/MI_LG,jumlah_murid_SD/MI_L,jumlah_murid_SD/MI_P,jumlah_murid_SD/MI_LP,jumlah_murid_SD/MI_L_Diperiksa,%jumlah_murid_SD/MI_L_Diperiksa,jumlah_murid_SD/MI_P_Diperiksa,%jumlah_murid_SD/MI_P_Diperiksa,jumlah_murid_SD/MI_LP_Diperiksa,%jumlah_murid_SD/MI_LP_Diperiksa,jumlah_murid_SD/MI_L_PR,jumlah_murid_SD/MI_P_PR,jumlah_murid_SD/MI_LP_PR,jumlah_murid_SD/MI_L_MP,%jumlah_murid_SD/MI_L_MP,jumlah_murid_SD/MI_P_MP,%jumlah_murid_SD/MI_P_MP,jumlah_murid_SD/MI_LP_MP,%jumlah_murid_SD/MI_LP_MP
0,CISEMPUR,12,1,8.3,12,100.0,1902,1698,3600,297,15.6,261,15.4,558,15.5,20,12,32,10,50.0,6,50.0,16,50.0
1,CIMANGGUNG,20,5,25.0,15,75.0,3437,3288,6725,1676,48.8,1634,49.7,3310,49.2,119,112,231,81,68.1,69,61.6,150,64.9
2,SAWAHDADAP,12,0,0.0,12,100.0,1676,1478,3154,292,17.4,251,17.0,543,17.2,35,41,76,12,34.3,22,53.7,34,44.7
3,TANJUNGSARI,20,2,10.0,20,100.0,437,441,878,385,88.1,390,88.4,775,88.3,40,41,81,30,75.0,35,85.4,65,80.2
4,MARGAJAYA,15,15,100.0,15,100.0,2166,2407,4573,845,39.0,781,32.4,1626,35.6,212,170,382,150,70.8,113,66.5,263,68.8
5,SUKASARI,12,10,83.3,10,83.3,1071,1607,2678,433,40.4,676,42.1,1109,41.4,431,674,1105,144,33.4,168,24.9,312,28.2
6,HAURNGOMBONG,15,0,0.0,15,100.0,1629,1544,3173,577,35.4,531,34.4,1108,34.9,98,80,178,90,91.8,80,100.0,170,95.5
7,PAMULIHAN,14,1,7.1,12,85.7,1127,1328,2455,258,22.9,226,17.0,484,19.7,128,169,297,104,81.3,117,69.2,221,74.4
8,RANCAKALONG,29,22,75.9,29,100.0,916,1520,2436,294,32.1,296,19.5,590,24.2,72,169,241,56,77.8,116,68.6,172,71.4
9,SUMEDANG SELATANSUMEDANG SELATAN,34,1,2.9,34,100.0,3175,2982,6157,2860,90.1,2719,34.4,1108,90.6,772,669,1441,587,76.0,520,77.7,1107,76.8


In [247]:
jawabarat_row = df[df['Kabupaten/Kota'] != 'Jawa Barat']
total_1 = jawabarat_row['jumlah_SD/MI'].sum()
total_2 = jawabarat_row['jumlah_SD/MI_SG_Massal'].sum()
total_3 = jawabarat_row['jumlah_SD/MI_LG'].sum()
total_4 = jawabarat_row['jumlah_murid_SD/MI_L'].sum()
total_5 = jawabarat_row['jumlah_murid_SD/MI_P'].sum()
total_6 = jawabarat_row['jumlah_murid_SD/MI_LP'].sum()
total_7 = jawabarat_row['jumlah_murid_SD/MI_L_Diperiksa'].sum()
total_8 = jawabarat_row['jumlah_murid_SD/MI_P_Diperiksa'].sum()
total_9 = jawabarat_row['jumlah_murid_SD/MI_LP_Diperiksa'].sum()
total_10 = jawabarat_row['jumlah_murid_SD/MI_L_PR'].sum()
total_11 = jawabarat_row['jumlah_murid_SD/MI_P_PR'].sum()
total_12 = jawabarat_row['jumlah_murid_SD/MI_LP_PR'].sum()
total_13 = jawabarat_row['jumlah_murid_SD/MI_L_MP'].sum()
total_14 = jawabarat_row['jumlah_murid_SD/MI_P_MP'].sum()
total_15 = jawabarat_row['jumlah_murid_SD/MI_LP_MP'].sum()

# Ganti nilai pada baris dengan Kabupaten/Kota == 'Jawa Barat'
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_SD/MI'] = total_1
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_SD/MI_SG_Massal'] = total_2
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_SD/MI_SG_Massal'] = round(total_2/total_1,3) * 100
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_SD/MI_LG'] = total_3
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_SD/MI_LG'] = round(total_3/total_2,3) * 100
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_L'] = total_4
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_P'] = total_5
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_LP'] = total_6
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_L_Diperiksa'] = total_7
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_murid_SD/MI_L_Diperiksa'] = round(total_7/total_4,3) * 100
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_P_Diperiksa'] = total_8
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_murid_SD/MI_P_Diperiksa'] = round(total_8/total_5,3)* 100
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_LP_Diperiksa'] = total_9
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_murid_SD/MI_LP_Diperiksa'] = round(total_9/total_6,3)* 100
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_L_PR'] = total_10
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_P_PR'] = total_11
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_LP_PR'] = total_12
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_L_MP'] = total_13
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_murid_SD/MI_L_MP'] = round(total_13/total_10,3)* 100
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_P_MP'] = total_14
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_murid_SD/MI_P_MP'] = round(total_14/total_11,3)* 100
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'jumlah_murid_SD/MI_LP_MP'] = total_15
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', '%jumlah_murid_SD/MI_LP_MP'] = round(total_15/total_12,3)* 100
df

,Kabupaten/Kota,jumlah_SD/MI,jumlah_SD/MI_SG_Massal,%jumlah_SD/MI_SG_Massal,jumlah_SD/MI_LG,%jumlah_SD/MI_LG,jumlah_murid_SD/MI_L,jumlah_murid_SD/MI_P,jumlah_murid_SD/MI_LP,jumlah_murid_SD/MI_L_Diperiksa,%jumlah_murid_SD/MI_L_Diperiksa,jumlah_murid_SD/MI_P_Diperiksa,%jumlah_murid_SD/MI_P_Diperiksa,jumlah_murid_SD/MI_LP_Diperiksa,%jumlah_murid_SD/MI_LP_Diperiksa,jumlah_murid_SD/MI_L_PR,jumlah_murid_SD/MI_P_PR,jumlah_murid_SD/MI_LP_PR,jumlah_murid_SD/MI_L_MP,%jumlah_murid_SD/MI_L_MP,jumlah_murid_SD/MI_P_MP,%jumlah_murid_SD/MI_P_MP,jumlah_murid_SD/MI_LP_MP,%jumlah_murid_SD/MI_LP_MP
0,CISEMPUR,12,1,8.3,12,100.0,1902,1698,3600,297,15.6,261,15.4,558,15.5,20,12,32,10,50.0,6,50.0,16,50.0
1,CIMANGGUNG,20,5,25.0,15,75.0,3437,3288,6725,1676,48.8,1634,49.7,3310,49.2,119,112,231,81,68.1,69,61.6,150,64.9
2,SAWAHDADAP,12,0,0.0,12,100.0,1676,1478,3154,292,17.4,251,17.0,543,17.2,35,41,76,12,34.3,22,53.7,34,44.7
3,TANJUNGSARI,20,2,10.0,20,100.0,437,441,878,385,88.1,390,88.4,775,88.3,40,41,81,30,75.0,35,85.4,65,80.2
4,MARGAJAYA,15,15,100.0,15,100.0,2166,2407,4573,845,39.0,781,32.4,1626,35.6,212,170,382,150,70.8,113,66.5,263,68.8
5,SUKASARI,12,10,83.3,10,83.3,1071,1607,2678,433,40.4,676,42.1,1109,41.4,431,674,1105,144,33.4,168,24.9,312,28.2
6,HAURNGOMBONG,15,0,0.0,15,100.0,1629,1544,3173,577,35.4,531,34.4,1108,34.9,98,80,178,90,91.8,80,100.0,170,95.5
7,PAMULIHAN,14,1,7.1,12,85.7,1127,1328,2455,258,22.9,226,17.0,484,19.7,128,169,297,104,81.3,117,69.2,221,74.4
8,RANCAKALONG,29,22,75.9,29,100.0,916,1520,2436,294,32.1,296,19.5,590,24.2,72,169,241,56,77.8,116,68.6,172,71.4
9,SUMEDANG SELATANSUMEDANG SELATAN,34,1,2.9,34,100.0,3175,2982,6157,2860,90.1,2719,34.4,1108,90.6,772,669,1441,587,76.0,520,77.7,1107,76.8
